In [1]:
import pandas as pd
import numpy as np
import random
import itertools

In [51]:
class Card:
    SUIT_MAP = {
        'D': 'Diamonds',
        'S': 'Spades',
        'C': 'Clubs',
        'H': 'Hearts'
    }

    RANK_MAP = {
        '2': "Two",
        '3': "Three",
        '4': "Four",
        '5': "Five",
        '6': "Six",
        '7': "Seven",
        '8': "Eight",
        '9': "Nine",
        '10': "Ten",
        'J': "Jack",
        'Q': "Queen",
        'K': "King",
        'A': "Ace"
    }

    COLOR_MAP = {
        'R': "Red",
        'B': "Black"
    }

    COLOR_SUIT_MAP = {
        'R':['H', 'D'],
        'B': ['S', 'C']
    }

    SUIT_COLOR_MAP = {
        'H': 'R',
        'D': 'R',
        'S': 'B',
        'C': 'B'
    }

    def __init__(self, suit: str, rank: str):
        self.suit = suit
        self.color = Card.SUIT_COLOR_MAP[suit]
        self.rank = rank

    def getSuit(self) -> str:
        return self.suit

    def getColor(self) -> str:
        return self.color

    def getRank(self) -> str:
        return self.rank

    def printCard(self):
        name = Card.RANK_MAP[self.rank] + " of " + Card.SUIT_MAP[self.suit]
        print(name)

    def getCard(self):
        return (self.rank + self.suit)

    def getShorthand(self):
        return self.rank + self.suit

    @classmethod
    # TODO fix for 10
    def fromShorthand(cls, shorthand: str):
        suit = shorthand[1]
        rank = shorthand[0]
        return cls(suit,rank)

In [38]:
class Deck:

    SUITS = ['S', 'H', 'D', 'C']
    
    RANKS = {
        '2': 2,
        '3': 3,
        '4': 4,
        '5': 5,
        '6': 6,
        '7': 7,
        '8': 8,
        '9': 9,
        '10': 10,
        'J': 11,
        'Q': 12,
        'K': 13,
        'A': 14
    }

    COLORS = ['R','B']

    def __init__(self):
        deck = list()
        for rank in Deck.RANKS:
            for suit in Deck.SUITS:
                deck.append(Card(suit,rank))
        random.shuffle(deck)
        self.deck = deck
        self.size = len(deck)

    def getDeck(self):
        return self.deck

    def drawCards(self, numberOfDraws: int = 1) -> list:

        assert self.size >= numberOfDraws, "Deck doesn't have enough cards!"

        cardsDrawn = []
        for i in range(numberOfDraws):
            cardsDrawn.append(self.deck.pop(-1))
            self.size = self.size - 1
        
        return cardsDrawn

    def shuffleDeck(self):
        random.shuffle(self.deck)



In [13]:
class Hand:

    def __init__(self, deck: Deck, numberOfCards: int = 3):
        self.hand = deck.drawCards(numberOfCards)

    def getHand(self):
        return self.hand
        
    

In [14]:
def getHandRankings():    
    rankings = {}
    suits = Deck.SUITS
    colors = Deck.COLORS
    ranks = Deck.RANKS
    ranksInv = {val:key for key,val in ranks.items()}

    rank = 1

    existingHands = set()

    # Three of a kind (trail)
    for item in list(ranks.keys())[-1::-1]:
        cards = [(item+suit) for suit in suits]
        members = list(itertools.combinations(cards,3))
        for member in members:
            hand = frozenset(member)
            rankings[hand] = rank
            existingHands.add(hand)

        rank+=1

    # Straight flush (pure sequence)
    for item in ['A']:
        for suit in suits:
            cards = [(item2+suit) for item2 in ['A','K','Q']]
            hand = frozenset(cards)
            rankings[hand] = rank
            existingHands.add(hand)

        rank+=1
        for suit in suits:
            cards = [(item2+suit) for item2 in ['A','2','3']]
            hand = frozenset(cards)
            rankings[hand] = rank
            existingHands.add(hand)

        rank+=1
    for item in range(13,3,-1):
        for suit in suits:
            cards = [ranksInv[item]+suit, ranksInv[item-1]+suit, ranksInv[item-2]+suit]
            hand = frozenset(cards)
            rankings[hand] = rank
            existingHands.add(hand)
        rank+=1


    # Straight (sequence)
    for item in ['A']:
        one = ['A'+suit for suit in suits]
        two = ['K'+suit for suit in suits]
        three = ['Q'+suit for suit in suits]
        full = [one,two,three]
        cards = set(itertools.product(*full))
        cards = {frozenset(item2) for item2 in cards}
        cards = cards.difference(existingHands)

        for hand in cards:
    
            rankings[frozenset(hand)] = rank
            existingHands.add(frozenset(hand))

        rank+=1


        one = ['A'+suit for suit in suits]
        two = ['2'+suit for suit in suits]
        three = ['3'+suit for suit in suits]
        full = [one,two,three]
        cards = frozenset(itertools.product(*full))
        cards = {frozenset(item2) for item2 in cards}
        cards = cards.difference(existingHands)

        for hand in cards:
            rankings[frozenset(hand)] = rank
            existingHands.add(frozenset(hand))
        rank+=1

    for item in range(13,3,-1):
        one = [ranksInv[item]+suit for suit in suits]
        two = [ranksInv[item-1]+suit for suit in suits]
        three = [ranksInv[item-2]+suit for suit in suits]
        full = [one, two, three]
        cards = frozenset(itertools.product(*full))
        cards = {frozenset(item2) for item2 in cards}
        cards = cards.difference(existingHands)
        
        for hand in cards:
            rankings[frozenset(hand)] = rank
            existingHands.add(frozenset(hand))
        rank+=1

    # Flush (color)
    for top in range(14,4, -1):
        for mid in range(top-1, 2, -1):
            if top-mid==1:
                for low in range(mid-2, 1, -1):
                    for suit in suits:
                        cards=[ranksInv[top]+suit, ranksInv[mid]+suit, ranksInv[low]+suit]
                        hand = frozenset(cards)
                        rankings[hand] = rank
                        rank+=1
                        existingHands.add(hand)
            else:
                for low in range(mid-1, 1, -1):
                    if ((top==14 and mid==3) and (low==2)):
                        continue
                    else:
                        for suit in suits:
                            cards=[ranksInv[top]+suit, ranksInv[mid]+suit, ranksInv[low]+suit]
                            hand = frozenset(cards)
                            rankings[hand] = rank
                            rank+=1
                            existingHands.add(hand)

    # Pair
    for pair in range(14,1, -1):
        for top in range(14,1, -1):
            if pair==top:
                continue
            else:
                cards = [(ranksInv[pair]+suit) for suit in suits]
                oneTwo = list(itertools.combinations(cards,2))
                three = [ranksInv[top]+suit for suit in suits]
                full = [oneTwo, three]
                cards = list(itertools.product(*full))
                cards = frozenset([(item[0][0],item[0][1],item[1]) for item in cards])
                for hand in cards:
                    hand = frozenset(hand)
                    rankings[hand] = rank
                    rank+=1
                    existingHands.add(hand)

    # No pair (high card)
    for top in range(14,4, -1):
        for mid in range(top-1, 2, -1):
            if top-mid==1:
                for low in range(mid-2, 1, -1):
                    one = [ranksInv[top]+suit for suit in suits]
                    two = [ranksInv[mid]+suit for suit in suits]
                    three = [ranksInv[low]+suit for suit in suits]
                    full = [one,two,three]
                    cards = frozenset(itertools.product(*full))
                    cards = {frozenset(item2) for item2 in cards}
                    cards = cards.difference(existingHands)
                    for hand in cards:
                        hand = frozenset(hand)
                        rankings[hand] = rank
                        existingHands.add(hand)
                    rank+=1
                        
            else:
                for low in range(mid-1, 1, -1):
                    if ((top==14 and mid==3) and (low==2)):
                        continue
                    else:
                        one = [ranksInv[top]+suit for suit in suits]
                        two = [ranksInv[mid]+suit for suit in suits]
                        three = [ranksInv[low]+suit for suit in suits]
                        full = [one,two,three]
                        cards = frozenset(itertools.product(*full))
                        cards = {frozenset(item2) for item2 in cards}
                        cards = cards.difference(existingHands)
                        for hand in cards:
                            hand = frozenset(hand)
                            rankings[hand] = rank
                            existingHands.add(hand)
                        rank+=1

    return rankings


In [39]:
# TODO demonstration

deck = Deck()

print("Drawing 20 cards")
drawn = []
left = []
for item in deck.drawCards(20):
    drawn.append(item.getCard())
for item in deck.getDeck():
    left.append(item.getCard())
print("Deck size remaining", deck.size)
print("Cards drawn:", drawn)
print("Deck remaining:", left)
print()

print("Drawing a hand of 3 cards")
hand = Hand(deck)
handCards = []
left = []
for card in hand.getHand():
    handCards.append(card.getCard())
for item in deck.getDeck():
    left.append(item.getCard())
print("Deck size remaining", deck.size)
print("Cards in hand:", handCards)
print("Deck remaining:", left)
print()

Drawing 20 cards
Deck size remaining 32
Cards drawn: ['10D', '10S', '7S', 'KH', '3S', 'QS', '8H', '2D', '3C', 'AH', 'AC', '7C', '2S', 'JC', '3H', '8S', '9C', '7D', '7H', 'AS']
Deck remaining: ['QD', '9S', 'QC', '4D', '10H', '4C', 'JH', '4H', '3D', 'QH', '8C', '6C', 'KD', '5C', 'AD', '5D', '5H', '9D', '10C', '6D', 'JS', '2C', '6H', '4S', 'KC', '9H', '8D', '6S', '5S', 'KS', 'JD', '2H']

Drawing a hand of 3 cards
Deck size remaining 29
Cards in hand: ['2H', 'JD', 'KS']
Deck remaining: ['QD', '9S', 'QC', '4D', '10H', '4C', 'JH', '4H', '3D', 'QH', '8C', '6C', 'KD', '5C', 'AD', '5D', '5H', '9D', '10C', '6D', 'JS', '2C', '6H', '4S', 'KC', '9H', '8D', '6S', '5S']



In [151]:
class Game:

    HAND_RANKINGS = getHandRankings()

    def __init__(self, noOfPlayers: int, jokers: list = None, handSize: int = 3):
        self.noOfPlayers = noOfPlayers
        self.deck = Deck()
        self.jokers = jokers
        self.handSize = handSize
        self.hands = self.generateHands()
        #self.winner = None
        #self.losers = None
        #self.tie = None

    def generateHands(self):
        self.deck.shuffleDeck()
        hands = []
        handsShorthand = []
        for _ in range(self.noOfPlayers):
            hands.append([])
        for _ in range(self.handSize):
            for i in range(self.noOfPlayers):
                hands[i].append(self.deck.drawCards(1)[0])

        for i in range(self.noOfPlayers):
            hands[i] = frozenset([item.getShorthand() for item in hands[i]])

        return hands

    def simulateGame(self):
        results = {item:np.array([0,0,1]) for item in self.hands}
        ranks = np.array([Game.HAND_RANKINGS[hand] for hand in self.hands])
        minIndices = np.where(ranks == ranks.min())[0]
        if len(minIndices)==1:
            win=1
            tie=0
        else:
            win=0
            tie=1
        for item in minIndices:
            results[self.hands[item]]=np.array([win,tie,1])
        return results


In [167]:
#simulations = {item:np.array([0,0,0]) for item in Game.HAND_RANKINGS.keys()}

for i in range(1000000):
    game = Game(8)
    res = game.simulateGame()
    for item in res:
        simulations[item]=np.add(simulations[item],res[item])

In [165]:
simulations

{frozenset({'AD', 'AH', 'AS'}): array([375,   0, 375]),
 frozenset({'AC', 'AH', 'AS'}): array([322,   0, 322]),
 frozenset({'AC', 'AD', 'AS'}): array([357,   0, 357]),
 frozenset({'AC', 'AD', 'AH'}): array([380,   0, 380]),
 frozenset({'KD', 'KH', 'KS'}): array([382,   0, 383]),
 frozenset({'KC', 'KH', 'KS'}): array([341,   0, 342]),
 frozenset({'KC', 'KD', 'KS'}): array([357,   0, 359]),
 frozenset({'KC', 'KD', 'KH'}): array([365,   0, 366]),
 frozenset({'QD', 'QH', 'QS'}): array([386,   0, 386]),
 frozenset({'QC', 'QH', 'QS'}): array([383,   0, 385]),
 frozenset({'QC', 'QD', 'QS'}): array([353,   0, 355]),
 frozenset({'QC', 'QD', 'QH'}): array([398,   0, 399]),
 frozenset({'JD', 'JH', 'JS'}): array([363,   0, 366]),
 frozenset({'JC', 'JH', 'JS'}): array([352,   0, 356]),
 frozenset({'JC', 'JD', 'JS'}): array([366,   0, 366]),
 frozenset({'JC', 'JD', 'JH'}): array([329,   0, 330]),
 frozenset({'10D', '10H', '10S'}): array([395,   0, 397]),
 frozenset({'10C', '10H', '10S'}): array([368

In [166]:
for item in simulations:
    print(item,simulations[item],round(simulations[item][0]/simulations[item][2],2))

frozenset({'AH', 'AD', 'AS'}) [375   0 375] 1.0
frozenset({'AH', 'AC', 'AS'}) [322   0 322] 1.0
frozenset({'AC', 'AD', 'AS'}) [357   0 357] 1.0
frozenset({'AH', 'AC', 'AD'}) [380   0 380] 1.0
frozenset({'KS', 'KH', 'KD'}) [382   0 383] 1.0
frozenset({'KS', 'KH', 'KC'}) [341   0 342] 1.0
frozenset({'KS', 'KC', 'KD'}) [357   0 359] 0.99
frozenset({'KH', 'KC', 'KD'}) [365   0 366] 1.0
frozenset({'QS', 'QD', 'QH'}) [386   0 386] 1.0
frozenset({'QS', 'QC', 'QH'}) [383   0 385] 0.99
frozenset({'QS', 'QD', 'QC'}) [353   0 355] 0.99
frozenset({'QD', 'QC', 'QH'}) [398   0 399] 1.0
frozenset({'JS', 'JD', 'JH'}) [363   0 366] 0.99
frozenset({'JS', 'JH', 'JC'}) [352   0 356] 0.99
frozenset({'JS', 'JD', 'JC'}) [366   0 366] 1.0
frozenset({'JD', 'JH', 'JC'}) [329   0 330] 1.0
frozenset({'10S', '10D', '10H'}) [395   0 397] 0.99
frozenset({'10S', '10C', '10H'}) [368   0 371] 0.99
frozenset({'10S', '10C', '10D'}) [389   0 392] 0.99
frozenset({'10C', '10D', '10H'}) [341   0 342] 1.0
frozenset({'9S', '9H